In [38]:
import pandas as pd

storms = range(1,97)
start_end = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project/tracks_square_storm/storms_with_steps_and_index.csv')

for storm in storms:

    master_1h = pd.read_csv(f'/Users/fabienaugsburger/Documents/GitHub/master-project/tracks_square_storm/ALL_TRACKS/tracks_1h/storm_{storm}.csv')
    #csv = pd.read_csv(f'/Users/fabienaugsburger/Documents/GitHub/master-project/datasets_3h/2m_dewpoint_temperature/storm_{storm}/max_{storm}_0.csv')

    timesteps_correct = start_end['1_hour_steps'][storm-1]+1
    timesteps_master = master_1h.shape[0]
    #timesteps_master = csv.shape[0]

    if timesteps_correct != timesteps_master:
        print(f'Error in storm {storm} with {timesteps_correct} steps and {timesteps_master} steps in master file')

In [39]:
# retrieve original storm tracks

storm_C3S = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project/tracks_square_storm/C3S_StormTracks_era5_19792021_0100_v3.csv',
                        delimiter=';')

In [40]:
# Create an empty list to store the storm dataframes
storm_dataframes = []
df = storm_C3S

# Split the dataframe whenever the first column starts with 'TRACK_ID'
split_indices = df.index[df.iloc[:, 0].str.startswith('TRACK_ID', na=False)].tolist()
split_indices.append(len(df))  # Add the last index to capture the final segment

# Generate the new dataframes
for i in range(len(split_indices) - 1):
    start_idx = split_indices[i]
    end_idx = split_indices[i + 1]
    storm_df = df.iloc[start_idx:end_idx].reset_index(drop=True)
    locals()[f'storm_{i + 1}'] = storm_df
    #storm_dataframes.append(storm_df)

#for idx, storm_df in enumerate(storm_dataframes):
#   tools.display_dataframe_to_user(name=f"storm_{idx + 1}", dataframe=storm_df)

In [8]:
# save the new storm tracks

'''for i in range(1,97):
    locals()[f'storm_{i}'].to_csv(f'/Users/fabienaugsburger/Documents/GitHub/master-project/tracks_square_storm/ALL_TRACKS/storm_{i}.csv', index=False)'''

In [41]:
# create 4 new columns named lon_east, lon_west, lat_north and lat_south
degree = 4

for i in range(1,97):

    # remove the 2 first rows of every storm
    locals()[f'storm_{i}'] = locals()[f'storm_{i}'].iloc[2:].reset_index(drop=True)

    # split the first column into 3 columns names Date, Longitude and Latitude
    locals()[f'storm_{i}'][['Date', 'Longitude', 'Latitude','Vorticity']] = locals()[f'storm_{i}']['Time&Longitude&Latitude'].str.split(' ', expand=True)
    locals()[f'storm_{i}'] = locals()[f'storm_{i}'].drop(columns=['Time&Longitude&Latitude'])

    # move the columns Date, Longitude and Latitude to the first columns
    cols = locals()[f'storm_{i}'].columns.tolist()
    cols = cols[-4:] + cols[:-4]
    locals()[f'storm_{i}'] = locals()[f'storm_{i}'][cols]
    
    # replace each value = 1e25 by NaN
    locals()[f'storm_{i}'] = locals()[f'storm_{i}'].replace(1e+25, float('nan'))

    # remove all columns expect Longitude and Latitude
    locals()[f'storm_{i}'] = locals()[f'storm_{i}'][['Longitude', 'Latitude']]

    locals()[f'storm_{i}']['lon_east'] = locals()[f'storm_{i}']['Longitude'].astype(float) - degree
    locals()[f'storm_{i}']['lon_west'] = locals()[f'storm_{i}']['Longitude'].astype(float) + degree
    locals()[f'storm_{i}']['lat_north'] = locals()[f'storm_{i}']['Latitude'].astype(float) + degree
    locals()[f'storm_{i}']['lat_south'] = locals()[f'storm_{i}']['Latitude'].astype(float) - degree

    # drop the columns Longitude and Latitude
    locals()[f'storm_{i}'] = locals()[f'storm_{i}'].drop(columns=['Longitude', 'Latitude'])

    # if the values in lon_east and lon_west are > 360, subtract 360
    locals()[f'storm_{i}']['lon_east'] = locals()[f'storm_{i}']['lon_east'].apply(lambda x: x - 360 if x > 360 else x)
    locals()[f'storm_{i}']['lon_east'] = locals()[f'storm_{i}']['lon_east'].apply(lambda x: 360 + x if x < 0 else x)
    locals()[f'storm_{i}']['lon_west'] = locals()[f'storm_{i}']['lon_west'].apply(lambda x: x - 360 if x > 360 else x)
    locals()[f'storm_{i}']['lon_west'] = locals()[f'storm_{i}']['lon_west'].apply(lambda x: 360 + x if x < 0 else x)

    # round the values in lon_east and lon_west to 6 decimals
    locals()[f'storm_{i}']['lon_east'] = locals()[f'storm_{i}']['lon_east'].round(6)
    locals()[f'storm_{i}']['lon_west'] = locals()[f'storm_{i}']['lon_west'].round(6)
    locals()[f'storm_{i}']['lat_north'] = locals()[f'storm_{i}']['lat_north'].round(6)
    locals()[f'storm_{i}']['lat_south'] = locals()[f'storm_{i}']['lat_south'].round(6)

    # save the new storm tracks

    locals()[f'storm_{i}'].to_csv(f'/Users/fabienaugsburger/Documents/GitHub/master-project/tracks_square_storm/ALL_TRACKS/tracks_3h/storm_{i}.csv', index=False)